<a href="https://colab.research.google.com/github/buggytanmoy77/Neurothon/blob/main/Model(with_W%26B).ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Installation of Pre-Requisite Libraries

In [ ]:
!pip install wandb scikit-learn pandas

import pandas as pd
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity
from ast import literal_eval
import wandb
import joblib

# Initialize W&B

In [ ]:
wandb.init(project="disease-detector", config={
    "model_type": "TF-IDF-Cosine",
    "rare_penalty": 0.2,
    "top_n": 5,
    "internal_n": 20,
    "max_questions": 5
})

### Before running the code below, make sure to have the csv file( from Database_Creation.ipynb) on your Colab

# Load data and vectorizer

In [ ]:
df = pd.read_csv('prefinal_diseases_with_symptoms_enhanced.csv')
df['Symptoms'] = df['Symptoms'].apply(lambda x: literal_eval(x) if isinstance(x, str) else [])

vectorizer = TfidfVectorizer()
symptom_vectors = vectorizer.fit_transform(df['Symptoms'].apply(lambda x: ' '.join(x)))
